In [5]:
import pandas as pd

ZiCan= r'E:\网优工作\BaseInformation\资管信息导出0915.csv'

HuaCan = r'E:\网优工作\BaseInformation\华为工参0911.csv'

Info = r'E:\网优工作\BaseInformation\天馈整理.xlsx'

CellList=r'Desktop\工作簿1.xlsx'

#相关文件导入
antinfo= pd.read_csv(ZiCan,usecols=['所属小区','挂高','方向角','所属机房/资源点','机械倾角','总倾角'],encoding='GBK')
antinfo.rename(columns={'所属小区':'小区CI',"所属机房/资源点":"机房"},inplace=True)
siteinfo = pd.read_csv(HuaCan,usecols=['ECGI','县市','最新网元名称','基站标识（ENODEBID）','小区标识','频段'],encoding='gbk')
siteinfo['小区CI']=256*siteinfo['基站标识（ENODEBID）'] + siteinfo['小区标识']
siteinfo.rename(columns={'县市':'区县','最新网元名称':'基站名称','ECGI':'CGI'},inplace=True)
S1=pd.read_excel(Info,'S1主控',usecols=['ECGI','最新网元名称'])
S1.rename(columns={'ECGI':'CGI','最新网元名称':'S1'},inplace=True)
highspeed=pd.read_excel(Info,'高速主控',usecols=['CGI','高速线路'])
contact=pd.read_excel(Info,'区域联系人')
Nopermit = pd.read_excel(Info,'无法调整小区',usecols=['小区CGI','无法调整原因'])
Nopermit.rename(columns={'小区CGI':'CGI'},inplace=True)
work=pd.read_excel(CellList,sheet_name=0,usecols=['CGI','备注'])

#小区信息匹配
cellinfo_an=pd.merge(siteinfo,antinfo,how='left',on='小区CI')
cellinfo_S1=pd.merge(cellinfo_an,S1,how='left',on='CGI')
cellinfo=pd.merge(cellinfo_S1,highspeed,how='left',on='CGI')
temp_work=pd.merge(work,cellinfo,how='left',on='CGI')
temp_work1=pd.merge(temp_work,contact,how='left',on='区县')
workfile=pd.merge(temp_work1,Nopermit,how='left',on='CGI')
workfile['派单原因']='重叠覆盖'

#设定列序
order=['区县','CGI','基站名称','机房','频段','小区CI','派单原因','挂高','方向角','总倾角','联系人','电话','备注','无法调整原因','S1','高速线路',"机械倾角"]
workfile=workfile[order]

#导出数据到EXCEL
workfile.to_excel(r'Desktop\天馈整理导出0915V1.xlsx',index=False)


当资管信息导出第二列有字符串数据时，程序会出错，有些数据匹配不出来，3DMIMO一般不能匹配不出来，需要先匹配NR小区号